In [ ]:
import pandas as pd
import os 
import requests
from dotenv import load_dotenv
load_dotenv()


def get_data(text, API_KEY):
  # Define the request parameters
  payload = {
      "textQuery": text
  }

  headers = {
      "Content-Type": "application/json",
      "X-Goog-Api-Key": API_KEY,
      "X-Goog-FieldMask": "places.name,places.id,places.types,places.formattedAddress,places.location,places.rating,places.userRatingCount,places.displayName,places.reviews"
  }

  # Send the POST request
  response = requests.post(
      "https://places.googleapis.com/v1/places:searchText", json=payload, headers=headers
  )

  # Check for successful response
  if response.status_code == 200:
    # Get the response data
    data = response.json()
  else:
    print(f"Error: {response.status_code}")
    raise ConnectionError(response.text)
  
  return data 



In [ ]:
paths = ['raw', 'data']

for path in paths:
    os.makedirs(f"{path}", exist_ok=True)

In [ ]:
# google places api query
query = "Restaurants in Abu Dhabi"

In [ ]:
# Replace 'API_KEY' with your actual Google Places API Key
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

data = get_data(query, API_KEY=API_KEY)

df = pd.DataFrame(data['places'])
df.to_parquet(f"./raw/{query}", index=False)

df['latitude'] = df.location.apply(lambda x: x['latitude'])
df['longitude'] = df.location.apply(lambda x: x['longitude'])
df  = df.drop('location', axis=1)
def process_reviews(reviews):
    return ' '.join((f"Rating: {review['rating']}\n{review['text']['text']}\n"
                        if "text" in review
                        else "") for review in reviews if review)

df['reviews'] = df['reviews'].apply(process_reviews)
df['query'] = query
df.to_parquet(f"./data/{query}", index=False)

In [ ]:

from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df[['rating', 'userRatingCount', 'reviews', 'query']], page_content_column="reviews")
documents = loader.load()
documents

In [ ]:
from langchain_together.embeddings import TogetherEmbeddings

embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import sqlite3
from langchain_chroma import Chroma

from utils.datasources import get_datasources

# check if data dir is empty
if len(get_datasources('./data')) == 1:
    # === to load first batch of documents into Chroma db vector store
    db = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory="./chroma_db")
else:
    # add more docs to persistant storage
    fromdisk = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
    fromdisk.add_documents(documents=documents)